In [2]:
!git clone https://github.com/instadeepai/multiomics-open-research.git


fatal: destination path 'multiomics-open-research' already exists and is not an empty directory.


In [ ]:
!pip install --quiet -e multiomics-open-research


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 MB 20.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 93.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 102.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 73.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.2 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00:00:01m00:01
   ━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pwd
!ls

In [ ]:
import sys
sys.path.append("/kaggle/working/multiomics-open-research")

from multiomics_open_research.bulk_rna_bert.pretrained import get_pretrained_model
from multiomics_open_research.bulk_rna_bert.preprocess import preprocess_rna_seq_for_bulkrnabert

In [ ]:

import haiku as hk
import jax
import jax.numpy as jnp
import pandas as pd


In [ ]:


# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name="bulk_rna_bert_tcga",
    embeddings_layers_to_save=(4,),
    checkpoint_directory="multiomics-open-research/checkpoints/",
)
forward_fn = hk.transform(forward_fn)



In [ ]:
# Get bulk RNASeq data and tokenize it
rna_seq_df = pd.read_csv("multiomics-open-research/data/tcga_sample.csv")
rna_seq_array = preprocess_rna_seq_for_bulkrnabert(rna_seq_df, config)
tokens_ids = tokenizer.batch_tokenize(rna_seq_array)
tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)



In [ ]:
tokens_df = pd.DataFrame(tokens) 
tokens_df["id"] = rna_seq_df["identifier"]
tokens_df.columns = rna_seq_df.columns
tokens_df = tokens_df.set_index("identifier")

tokens_df

In [ ]:
import numpy as np

sample_embeddings = []

for i in range(tokens.shape[0]):
    print(i)
    sample_tokens = tokens[i:i+1]  # Shape (1, seq_len)
    key = jax.random.PRNGKey(i)    # different key per sample
    out = forward_fn.apply(parameters, key, sample_tokens)
    emb = out["embeddings_4"].mean(axis=1)  # Shape (1, dim)
    sample_embeddings.append(np.array(emb[0]))

mean_embedding = np.stack(sample_embeddings)  # Shape (n_samples, embedding_dim)


In [ ]:
mean_embeddings

In [ ]:
import umap
import matplotlib.pyplot as plt

# Reduce embeddings to 2D
reducer = umap.UMAP(random_state=42)
proj = reducer.fit_transform(mean_embedding)

# Plot
plt.figure(figsize=(6, 4))
plt.scatter(proj[:, 0], proj[:, 1], s=100)

# Add labels
for i, label in enumerate(rna_seq_df["identifier"]):
    plt.text(proj[i, 0], proj[i, 1], label, fontsize=9)

plt.title("UMAP of BulkRNASeq BERT Embeddings")
plt.xlabel("UMAP 1")
plt.ylabel("UMAP 2")
plt.grid(True)
plt.show()
